<a href="https://colab.research.google.com/github/Warvito/Normative-modelling-using-deep-autoencoders/blob/master/notebooks/predict_deviation_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deviation scores using all trained models

Here in this notebook, we implemented an easy way to you try our normative models based on autoencoders trained on the UK Biobank data. **Disclaimer**: this script can not be used for clinical purposes.

Let's start!

---
## Enabling the GPU

First, you'll need to enable [GPUs](https://cloud.google.com/gpu) for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

## Load trained models
Next, we will load the trained normative models into this colab environment. In our study, we trained 1,000 different models on the UK BIobank using the resampling method called bootstrap method.

All the saved files are available at https://www.dropbox.com/s/bs89t2davs1p2dm/models_for_normative_paper_2019.zip?dl=0 .

This zipped file contains the models files created using the  bootstrap_train_aae_supervised.py script. The models files are organized in subdirectories where each one correspond to a bootstrap iteration. In each iteration, we stored the data scaler, the age and gender encoders, and the encoder and decoder of the normative model.

Besides the models, the zipped file contains two templates files (used later in this notebook).

In [0]:
!wget -O models.zip --no-check-certificate https://www.dropbox.com/s/bs89t2davs1p2dm/models_for_normative_paper_2019.zip?dl=0

--2019-12-04 16:42:10--  https://www.dropbox.com/s/bs89t2davs1p2dm/models_for_normative_paper_2019.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bs89t2davs1p2dm/models_for_normative_paper_2019.zip [following]
--2019-12-04 16:42:10--  https://www.dropbox.com/s/raw/bs89t2davs1p2dm/models_for_normative_paper_2019.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4a51b3038a85342edc1a67243d.dl.dropboxusercontent.com/cd/0/inline/Atn60gy3Ie-65D_7iYQUfRoiqy_knjTZUqAGyTO-L82Kq9QF9fh2wUGA9puU_TjpTiuBSfhszHNhDxec9ikTnObWziRRxYiFGISVA9bCm-CjMgCMduV_0kliQGqHrLIb52M/file# [following]
--2019-12-04 16:42:11--  https://uc4a51b3038a85342edc1a67243d.dl.dropboxusercontent.com/cd/0/inline/Atn60gy3Ie-65D_7iYQUfRoiqy_knjTZUqAGy

## Unzipping models files


In [2]:
!unzip models.zip

Archive:  models.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of models.zip or
        models.zip.zip, and cannot find models.zip.ZIP, period.


As showed below, in the Google colab environment, there is an arrow mark which looks like “>” at the left-hand side of the cells.

<img src="https://github.com/Warvito/Normative-modelling-using-deep-autoencoders/blob/master/notebooks/figures/files.png?raw=1">

When you click that you will find a tab with three options, just have to select “Files” to explore the loaded models. 

## Importing Python libraries


In [3]:
%tensorflow_version 2.x
from pathlib import Path
import warnings

import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from google.colab import files
from tqdm import tqdm

TensorFlow 2.x selected.


In [4]:
tf.__version__

'2.0.0'

## Downloading freesurferData.csv and participants.tsv templates
In order to make predictions of your data, it is necessary to make it in the format to correctly read by this script. To facilitate this process, we supply the template files to be filled with your data.

As shown below, these template files contain the names of the necessary columns to run the script.

In [5]:
pd.read_csv('freesurferData.csv')

,Participant_ID,EstimatedTotalIntraCranialVol,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,4th-Ventricle,Brain-Stem,Left-Hippocampus,Left-Amygdala,CSF,Left-Accumbens-area,Left-VentralDC,Right-Lateral-Ventricle,Right-Inf-Lat-Vent,Right-Cerebellum-White-Matter,Right-Cerebellum-Cortex,Right-Thalamus-Proper,Right-Caudate,Right-Putamen,Right-Pallidum,Right-Hippocampus,Right-Amygdala,Right-Accumbens-area,Right-VentralDC,CC_Posterior,CC_Mid_Posterior,CC_Central,CC_Mid_Anterior,CC_Anterior,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,...,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume,rh_bankssts_volume,rh_caudalanteriorcingulate_volume,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,rh_lateraloccipital_volume,rh_lateralorbitofrontal_volume,rh_lingual_volume,rh_medialorbitofrontal_volume,rh_middletemporal_volume,rh_parahippocampal_volume,rh_paracentral_volume,rh_parsopercularis_volume,rh_parsorbitalis_volume,rh_parstriangularis_volume,rh_pericalcarine_volume,rh_postcentral_volume,rh_posteriorcingulate_volume,rh_precentral_volume,rh_precuneus_volume,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,sub-X,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [6]:
pd.read_csv('participants.tsv', sep='\t')

,Participant_ID,Gender,Age
0,sub-X,0,60


* Note: The column with gender is codified as 0 = "Female" and 1 = "Male".

The next cells will start the download of the templates.

---



In [7]:
files.download('templates/freesurferData.csv')

FileNotFoundError: ignored

In [0]:
files.download('templates/participants.tsv')

After filled the templates, upload the files to the Google colab environment.

**Note: You can create the freesurferData.csv file using our colab script on this** [link](https://colab.research.google.com/github/Warvito/Normative-modelling-using-deep-autoencoders/blob/master/notebooks/freesurfer_organizer.ipynb).

Note2: Your data will only be loaded in this runtime of the Google colab. This code is being executed at the Google Cloud Platform by default, and you are not making your data available for our team. If you are concern about uploading your data to the Google Cloud Platform, please, consider executing this notebook in a local runtime in your computer (https://research.google.com/colaboratory/local-runtimes.html).

First, start uploading the freesurferData.csv.

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Then, upload the participants.tsv file.

In [0]:
uploaded = files.upload()

for fn2 in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn2, length=len(uploaded[fn2])))

In [0]:
freesurfer_data_df = pd.read_csv(fn)
participants_df = pd.read_csv(fn2, sep='\t')

dataset_df = pd.merge(freesurfer_data_df, participants_df, on='Participant_ID')

In [0]:
dataset_df

## Predicting the deviations
After loading the data, we predict the deviations of the new data based on our trained normative models.

We begin the processing by setting the random seeds.

In [0]:
# Set random seed
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

Next, we define the name of the brain regions in the variable COLUMNS_NAME.

In [0]:
#@title
COLUMNS_NAME = ['Left-Lateral-Ventricle',
                'Left-Inf-Lat-Vent',
                'Left-Cerebellum-White-Matter',
                'Left-Cerebellum-Cortex',
                'Left-Thalamus-Proper',
                'Left-Caudate',
                'Left-Putamen',
                'Left-Pallidum',
                '3rd-Ventricle',
                '4th-Ventricle',
                'Brain-Stem',
                'Left-Hippocampus',
                'Left-Amygdala',
                'CSF',
                'Left-Accumbens-area',
                'Left-VentralDC',
                'Right-Lateral-Ventricle',
                'Right-Inf-Lat-Vent',
                'Right-Cerebellum-White-Matter',
                'Right-Cerebellum-Cortex',
                'Right-Thalamus-Proper',
                'Right-Caudate',
                'Right-Putamen',
                'Right-Pallidum',
                'Right-Hippocampus',
                'Right-Amygdala',
                'Right-Accumbens-area',
                'Right-VentralDC',
                'CC_Posterior',
                'CC_Mid_Posterior',
                'CC_Central',
                'CC_Mid_Anterior',
                'CC_Anterior',
                'lh_bankssts_volume',
                'lh_caudalanteriorcingulate_volume',
                'lh_caudalmiddlefrontal_volume',
                'lh_cuneus_volume',
                'lh_entorhinal_volume',
                'lh_fusiform_volume',
                'lh_inferiorparietal_volume',
                'lh_inferiortemporal_volume',
                'lh_isthmuscingulate_volume',
                'lh_lateraloccipital_volume',
                'lh_lateralorbitofrontal_volume',
                'lh_lingual_volume',
                'lh_medialorbitofrontal_volume',
                'lh_middletemporal_volume',
                'lh_parahippocampal_volume',
                'lh_paracentral_volume',
                'lh_parsopercularis_volume',
                'lh_parsorbitalis_volume',
                'lh_parstriangularis_volume',
                'lh_pericalcarine_volume',
                'lh_postcentral_volume',
                'lh_posteriorcingulate_volume',
                'lh_precentral_volume',
                'lh_precuneus_volume',
                'lh_rostralanteriorcingulate_volume',
                'lh_rostralmiddlefrontal_volume',
                'lh_superiorfrontal_volume',
                'lh_superiorparietal_volume',
                'lh_superiortemporal_volume',
                'lh_supramarginal_volume',
                'lh_frontalpole_volume',
                'lh_temporalpole_volume',
                'lh_transversetemporal_volume',
                'lh_insula_volume',
                'rh_bankssts_volume',
                'rh_caudalanteriorcingulate_volume',
                'rh_caudalmiddlefrontal_volume',
                'rh_cuneus_volume',
                'rh_entorhinal_volume',
                'rh_fusiform_volume',
                'rh_inferiorparietal_volume',
                'rh_inferiortemporal_volume',
                'rh_isthmuscingulate_volume',
                'rh_lateraloccipital_volume',
                'rh_lateralorbitofrontal_volume',
                'rh_lingual_volume',
                'rh_medialorbitofrontal_volume',
                'rh_middletemporal_volume',
                'rh_parahippocampal_volume',
                'rh_paracentral_volume',
                'rh_parsopercularis_volume',
                'rh_parsorbitalis_volume',
                'rh_parstriangularis_volume',
                'rh_pericalcarine_volume',
                'rh_postcentral_volume',
                'rh_posteriorcingulate_volume',
                'rh_precentral_volume',
                'rh_precuneus_volume',
                'rh_rostralanteriorcingulate_volume',
                'rh_rostralmiddlefrontal_volume',
                'rh_superiorfrontal_volume',
                'rh_superiorparietal_volume',
                'rh_superiortemporal_volume',
                'rh_supramarginal_volume',
                'rh_frontalpole_volume',
                'rh_temporalpole_volume',
                'rh_transversetemporal_volume',
                'rh_insula_volume']

Then, we calculate the relative brain region volumes (original volume divided by the total intracranial volume). 

In [0]:
# Get the relative brain region volumes
x_dataset = dataset_df[COLUMNS_NAME].values

tiv = dataset_df['EstimatedTotalIntraCranialVol'].values
tiv = tiv[:, np.newaxis]

x_dataset = (np.true_divide(x_dataset, tiv)).astype('float32')

Next, we iterate over all models performing the calculation of the deviations.

* Note: if the age of the subject in lower than 47 or higher than 73, the age value will be clipped the be inside the range (47,73), for example, if someone has age = 40, it will be rounded to 47.


dataset.loc[(dataset['Age'] >= 47) & (dataset['Age'] <= 73)]

In [0]:
warnings.filterwarnings('ignore')

model_dir = Path('models')
N_BOOTSTRAP = 1000

# Create dataframe to store outputs
reconstruction_error_df = pd.DataFrame(columns=['Participant_ID'])
reconstruction_error_df['Participant_ID'] = dataset_df['Participant_ID']

# ----------------------------------------------------------------------------
for i_bootstrap in tqdm(range(N_BOOTSTRAP)):
    bootstrap_model_dir = model_dir / '{:03d}'.format(i_bootstrap)

    # ----------------------------------------------------------------------------
    encoder = keras.models.load_model(bootstrap_model_dir / 'encoder.h5', compile=False)
    decoder = keras.models.load_model(bootstrap_model_dir / 'decoder.h5', compile=False)

    scaler = joblib.load(bootstrap_model_dir / 'scaler.joblib')

    enc_age = joblib.load(bootstrap_model_dir / 'age_encoder.joblib')
    enc_gender = joblib.load(bootstrap_model_dir / 'gender_encoder.joblib')

    # ----------------------------------------------------------------------------
    x_normalized = scaler.transform(x_dataset)

    # ----------------------------------------------------------------------------
    age = dataset_df['Age'].values
    age = np.clip(age, 47, 73)
    age = age[:, np.newaxis].astype('float32')
    one_hot_age = enc_age.transform(age)

    gender = dataset_df['Gender'].values[:, np.newaxis].astype('float32')
    one_hot_gender = enc_gender.transform(gender)

    y_data = np.concatenate((one_hot_age, one_hot_gender), axis=1).astype('float32')

    # ----------------------------------------------------------------------------
    encoded = encoder(x_normalized, training=False)
    reconstruction = decoder(tf.concat([encoded, y_data], axis=1), training=False)

    # ----------------------------------------------------------------------------
    reconstruction_error = np.mean((x_normalized - reconstruction) ** 2, axis=1)

    reconstruction_error_df[('Reconstruction error {:03d}'.format(i_bootstrap))] = reconstruction_error

Finally, we compute the mean reconstruction error and save the file with the reconstruction errors.

In [0]:
reconstruction_error_df[reconstruction_error_df.columns[1:]]

In [0]:
reconstruction_error_df['Mean reconstruction error'] = reconstruction_error_df[reconstruction_error_df.columns[1:]].mean(axis=1)
reconstruction_error_df.to_csv('reconstruction_error.csv', index=False)

## Download predictions
Finally, you can download the result in the "Files" tab or executing the cell below.

In [0]:
files.download('reconstruction_error.csv')